In [5]:
import warnings
warnings.filterwarnings('ignore')

D:\env\anaconda3\envs\py27\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-22 14:34:24.610672. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
path = u'../9.Regression/iris.data'
data = pd.read_csv(path, header=None)
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
x, y = data[range(4)], data[4]
y = pd.Categorical(y).codes
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, \
                                                   test_size=50)

## xgboost训练

In [9]:
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'eval'), (data_train, 'train')]
param = {
    'max_depth': 4,
    'eta': 0.3,
    'silent': 1,
    'objective': 'multi:softmax',
    'num_class': 3
}

In [11]:
bst = xgb.train(param, data_train, num_boost_round=6, evals=watch_list)
y_hat = bst.predict(data_test)
result = y_test == y_hat
print '准确率:\t', result.mean()
print '准确率:\t', float(np.sum(result)) / len(y_hat)

[0]	eval-merror:0.02	train-merror:0.02
[1]	eval-merror:0.02	train-merror:0.02
[2]	eval-merror:0.02	train-merror:0.02
[3]	eval-merror:0.02	train-merror:0.02
[4]	eval-merror:0.02	train-merror:0.02
[5]	eval-merror:0.02	train-merror:0.02
准确率:	0.98
准确率:	0.98


## 其他模型

In [14]:
models = [
    ('LogisticRegression', LogisticRegressionCV(Cs=10, cv=3)),
    ('RandomForest', RandomForestClassifier(n_estimators=30, criterion='gini'))
]
for name, model in models:
    model.fit(x_train, y_train)
    print name, '训练集正确率: ',accuracy_score(y_train, model.predict(x_train))
    print name, '测试集正确率: ',accuracy_score(y_test, model.predict(x_test))

LogisticRegression 训练集正确率:  0.96
LogisticRegression 测试集正确率:  0.9
RandomForest 训练集正确率:  1.0
RandomForest 测试集正确率:  0.96
